### Resume Environment Setup

In [ ]:
USE ROLE accountadmin;

CREATE OR REPLACE DATABASE RESUME_SCREENING_DB;

CREATE OR REPLACE SCHEMA RESUME_AI_SCHEMA;

CREATE OR REPLACE WAREHOUSE RESUME_AI_WH;

CREATE OR REPLACE WAREHOUSE RESUME_AI_MEDIUM 
WAREHOUSE_SIZE = 'MEDIUM'  
AUTO_SUSPEND = 60;

CREATE OR REPLACE WAREHOUSE RESUME_AI_LARGE
WAREHOUSE_SIZE = 'LARGE'  
AUTO_SUSPEND = 60;

USE DATABASE resume_screening_db;

USE SCHEMA resume_ai_schema;

CREATE STAGE RESUME_DOCS
    DIRECTORY = ( ENABLE = TRUE ) 
    ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

CREATE ROLE doc_ai_role;

-- Step 2: Grant required privileges
GRANT USAGE ON WAREHOUSE resume_ai_wh TO ROLE doc_ai_role;
GRANT USAGE ON WAREHOUSE resume_ai_medium TO ROLE doc_ai_role;
GRANT USAGE ON WAREHOUSE resume_ai_large TO ROLE doc_ai_role;

GRANT USAGE ON DATABASE resume_screening_db TO ROLE doc_ai_role;
GRANT USAGE ON SCHEMA resume_ai_schema TO ROLE doc_ai_role;
-- GRANT CREATE SNOWFLAKE.ML.DOCUMENT_INTELLIGENCE ON SCHEMA doc_ai_db.doc_ai_schema TO ROLE doc_ai_role;
-- GRANT CREATE MODEL ON SCHEMA doc_ai_db.doc_ai_schema TO ROLE doc_ai_role;

-- Step 3: Grant the role to a specific user
GRANT ROLE doc_ai_role TO USER MENGTHONG01;

SELECT CURRENT_ROLE();







### Snowflake Git API Integration

In [ ]:
CREATE OR REPLACE SECRET myco_git_secret
  TYPE = password
  USERNAME = 'MengthongLong'
  PASSWORD = '';

CREATE OR REPLACE API INTEGRATION git_api_integration
  API_PROVIDER = git_https_api
  API_ALLOWED_PREFIXES = ('https://github.com/MengthongLong/resume-screening.git')
  ALLOWED_AUTHENTICATION_SECRETS = (myco_git_secret)
  ENABLED = TRUE;

SHOW SECRETS;
SHOW API INTEGRATIONS;
SHOW GIT REPOSITORIES;

  
DROP SECRET my_git_secret;
DROP API INTEGRATION my_api_integration;

//create git repository 
CREATE GIT REPOSITORY "resume-screening" 
	ORIGIN = 'https://github.com/MengthongLong/resume-screening.git' 
	API_INTEGRATION = 'GIT_API_INTEGRATION' 
	GIT_CREDENTIALS = 'RESUME_SCREENING_DB.RESUME_AI_SCHEMA.MYCO_GIT_SECRET';



### Resume Prediction From Document AI

In [ ]:
USE role accountadmin;
USE database resume_screening_db;
USE schema resume_ai_schema;

//create a raw json file of the reusme prediction 
CREATE OR REPLACE TABLE resume_prediction_json AS 
SELECT RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_SCREENING!PREDICT(
  GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, RELATIVE_PATH), 2) AS resume_prediction 
FROM DIRECTORY(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS);

//view sample json data
SELECT *
FROM resume_prediction_json;


-- Create a table with all values and scores
CREATE OR REPLACE TABLE resume_prediction AS
WITH temp AS(
-- First part gets the result from applying the model on the pdf documents as a JSON with additional metadata
    SELECT 
        Relative_path as file_name, 
        size as file_size, 
        last_modified, 
        file_url as snowflake_file_url,
        --VERIFY THAT BELOW IS USING THE SAME NAME AND NUMER AS THE MODEL INSTRUCTIONS YOU COPIED IN THE PREVIOUS STEP! 
        RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_SCREENING!PREDICT(GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, RELATIVE_PATH), 2) AS        
        resume_prediction
    from directory(@RESUME_DOCS)
)

SELECT 
    file_name, 
    file_size, 
    last_modified, 
    snowflake_file_url, 
    RESUME_PREDICTION:__documentMetadata.ocrScore::FLOAT AS ocrScore,
    RESUME_PREDICTION:"full_name"[0]:value::STRING AS full_name,
    RESUME_PREDICTION:"phone_number"[0]:value::STRING AS phone_number,
    RESUME_PREDICTION:"email_address"[0]:value::STRING AS email_address,
    RESUME_PREDICTION:"location"[0]:value::STRING AS location,
    RESUME_PREDICTION:"job_title"[0]:value::STRING AS job_title,
    RESUME_PREDICTION:"work_experience_roles" AS work_experience_roles,
    RESUME_PREDICTION:"work_experience_summary" AS recent_work_experience,
    RESUME_PREDICTION:"technical_skills" AS technical_skills,
    RESUME_PREDICTION:"year_of_experiences"[0]:value::INT AS year_of_experiences,
    RESUME_PREDICTION:"field_of_study"[0]:value::STRING AS field_of_study,
    RESUME_PREDICTION:"university_name"[0]:value::STRING AS university_name,
    RESUME_PREDICTION:"certifications" AS certifications,
    RESUME_PREDICTION:"languages_spoken" AS languages_spoken,
FROM temp;


//view sample prediciton data
SELECT *
FROM resume_prediction;



### Resume Prediction Reference Code

In [ ]:

USE ROLE accountadmin;

DROP ROLE doc_ai_role;

// Extract the resume prediciton result from resume_model stage 
SELECT RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_MODEL!PREDICT(
  GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, RELATIVE_PATH), 2) AS resume_prediction 
FROM DIRECTORY(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS);

//Create the resume prediction json table 
CREATE OR REPLACE TABLE resume_prediction_json AS 
SELECT RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_MODEL!PREDICT(
  GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, RELATIVE_PATH), 2) AS resume_prediction 
FROM DIRECTORY(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS);


SELECT *
from resume_prediction_json;

describe table resume_prediction_json;

-- SELECT DISTINCT
--     resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
--     resume_prediction:"email_address"[0]::variant:"value"::string AS email,
--     resume_prediction:"phone_number"[0]::variant:"value"::string AS phone,
--     resume_prediction:"location"[0]::variant:"value"::string AS location,
--     resume_prediction:"university_name"[0]::variant:"value"::string AS university_1,
--     resume_prediction:"university_name"[1]::variant:"value"::string AS university_2,
--     resume_prediction:"degree"[0]::variant:"value"::string AS degree,
--     resume_prediction:"field_of_study"[0]::variant:"value"::string AS field_of_study_1,
--     resume_prediction:"field_of_study"[1]::variant:"value"::string AS field_of_study_2,
--     resume_prediction:"work_experience_roles"[0]::variant:"value"::string AS job_role_1,
--     resume_prediction:"work_experience_roles"[1]::variant:"value"::string AS job_role_2,
--     resume_prediction:"work_experience_roles"[2]::variant:"value"::string AS job_role_3,
--     resume_prediction:"year_of_experiences"[0]::variant:"value"::string AS years_of_experience,
--     resume_prediction:"job_title"[0]::variant:"value"::string AS job_title,
--     resume_prediction:"languages_spoken"[0]::variant:"value"::string AS languages_spoken_score,
--     resume_prediction:"technical_skills"[0]::variant:"value"::string AS technical_skills_score,
--     resume_prediction:"work_experience_summary"[0]::variant:"value"::string AS work_experience_summary,
--     resume_prediction:"certifications"[0]::variant:"value"::string AS certifications,
--     resume_prediction:"__documentMetadata"::variant:"ocrScore"::float AS ocr_score
-- FROM resume_prediction_json;

//create the resume prediction table from json file
CREATE OR REPLACE table resume_prediction AS    
    WITH base AS (
        SELECT DISTINCT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name
        FROM resume_prediction_json
    ),
    
    tech_skills AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ', ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS technical_skills
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"technical_skills"))
        GROUP BY full_name
    ),
    
    work_summary AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ' || ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS work_experience_summary
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"work_experience_summary"))
        GROUP BY full_name
    ),
    
    certifications AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ', ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS certifications
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"certifications"))
        GROUP BY full_name
    ),
    
    degrees AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ', ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS degree
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"degree"))
        GROUP BY full_name
    ),

    field_of_study AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ', ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS field_of_study
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"field_of_study"))
        GROUP BY full_name
    ),
    
    job_titles AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ', ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS job_title
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"job_title"))
        GROUP BY full_name
    ),
    
    languages AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ', ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS languages_spoken
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"languages_spoken"))
        GROUP BY full_name
    ),
    
    work_experience_roles AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            LISTAGG(value:"value"::string, ', ') 
                WITHIN GROUP (ORDER BY value:"value"::string) AS work_experience_roles
        FROM resume_prediction_json, 
        TABLE(FLATTEN(input => resume_prediction:"work_experience_roles"))
        GROUP BY full_name
    ),

    year_of_experiences AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            flattened.value:"value"::string AS year_of_experiences
        FROM resume_prediction_json,
        LATERAL FLATTEN(input => resume_prediction:"year_of_experiences") AS flattened
        GROUP BY full_name, year_of_experiences
    ),
    
    phone_number AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            resume_prediction:"phone_number"[0]::variant:"value"::string AS phone_number
        FROM resume_prediction_json
        GROUP BY full_name, phone_number
    ),
    
    location AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            resume_prediction:"location"[0]::variant:"value"::string AS location
        FROM resume_prediction_json
        GROUP BY full_name, location
    ),
        
    university_name AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            resume_prediction:"university_name"[0]::variant:"value"::string AS university_name
        FROM resume_prediction_json
        GROUP BY full_name, university_name
    ),
    
    email AS (
        SELECT 
            resume_prediction:"full_name"[0]::variant:"value"::string AS full_name,
            resume_prediction:"email_address"[0]::variant:"value"::string AS email_address
        FROM resume_prediction_json
        GROUP BY full_name, email_address
    )
    
    SELECT DISTINCT
        -- Personal Info
        CURRENT_TIMESTAMP() AS extraction_timestamp,
        b.full_name,
        COALESCE(p.phone_number, '') AS phone_number,
        COALESCE(e.email_address, '') AS email_address,
        COALESCE(lc.location, '') AS location,
        
        -- Work
        COALESCE(j.job_title, '') AS job_title,
        COALESCE(wr.work_experience_roles, '') AS work_experience_roles,
        COALESCE(w.work_experience_summary, '') AS work_experience_summary,
        COALESCE(t.technical_skills, '') AS technical_skills,
        COALESCE(ye.year_of_experiences, '') AS year_of_experiences,
        
        -- Education
        COALESCE(d.degree, '') AS degree,
        COALESCE(fs.field_of_study, '') AS field_of_study,
        COALESCE(un.university_name, '') AS university_name,
        COALESCE(c.certifications, '') AS certifications,
        
        -- Languages
        COALESCE(l.languages_spoken, '') AS languages_spoken
        
    FROM base b
    LEFT JOIN tech_skills t ON b.full_name = t.full_name
    LEFT JOIN work_summary w ON b.full_name = w.full_name
    LEFT JOIN certifications c ON b.full_name = c.full_name
    LEFT JOIN degrees d ON b.full_name = d.full_name
    LEFT JOIN job_titles j ON b.full_name = j.full_name
    LEFT JOIN languages l ON b.full_name = l.full_name
    LEFT JOIN work_experience_roles wr ON b.full_name = wr.full_name
    LEFT JOIN phone_number p ON b.full_name = p.full_name
    LEFT JOIN location lc ON b.full_name = lc.full_name
    LEFT JOIN email e ON b.full_name = e.full_name
    LEFT JOIN field_of_study fs ON b.full_name = fs.full_name
    LEFT JOIN year_of_experiences ye ON b.full_name = ye.full_name
    LEFT JOIN university_name un ON b.full_name = un.full_name;

SELECT *
FROM resume_prediction;

DESC Table resume_prediction;

//Remove the files from the stage 
REMOVE @RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS;
//View the files in the stage
LIST @RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS;


//metadata json 

LIST @RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS;

CREATE OR REPLACE TABLE resume_prediction_json_test AS 
SELECT 
    CURRENT_TIMESTAMP() AS extraction_timestamp,
    resume_prediction
FROM (
    SELECT 
        RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_MODEL!PREDICT(
            GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, RELATIVE_PATH), 1
        ) AS resume_prediction
    FROM DIRECTORY(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS)
);


-- Create a table to store metadata information
CREATE OR REPLACE TABLE resume_metadata AS
SELECT DISTINCT
    METADATA$FILENAME AS file_name, 
    SPLIT_PART(METADATA$FILENAME, '.', -1) AS file_type, 
    METADATA$FILE_LAST_MODIFIED AS file_created
FROM @RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS;

//metada 
SELECT *
FROM resume_metadata;

//json file with extraction timestamp 
CREATE OR REPLACE TABLE resume_prediction_json_test AS 
SELECT 
    CURRENT_TIMESTAMP() AS extraction_timestamp,
    resume_prediction
FROM (
    SELECT 
        RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_MODEL!PREDICT(
            GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, RELATIVE_PATH), 1
        ) AS resume_prediction
    FROM DIRECTORY(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS)
);


//resume prediction with metadata

CREATE OR REPLACE TABLE resume_prediction_metadata AS 
SELECT 
    CURRENT_TIMESTAMP() AS extraction_timestamp,
    file_metadata.file_name,
    file_metadata.file_type,
    file_metadata.file_created,
    prediction_result.resume_prediction
FROM (
    -- Extract file metadata from the directory, filtering only relevant files (e.g., by extension or file age)
    SELECT 
        METADATA$FILENAME AS file_name,
        SPLIT_PART(METADATA$FILENAME, '.', -1) AS file_type,
        METADATA$FILE_LAST_MODIFIED AS file_created,
        GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, METADATA$FILENAME) AS presigned_url
    FROM @RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS
    WHERE METADATA$FILENAME LIKE '%.png'  -- Filter by file type if needed
    -- Add additional filtering conditions if necessary, such as excluding files already processed
) AS file_metadata
-- Use a regular JOIN instead of LATERAL if possible, or process predictions in batches
JOIN (
    -- Apply prediction model to each file using the presigned URL
    SELECT 
        METADATA$FILENAME AS file_name,
        RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_MODEL!PREDICT(
            GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, METADATA$FILENAME), 1
        ) AS resume_prediction
    FROM @RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS
    WHERE METADATA$FILENAME LIKE '%.png'
) AS prediction_result
ON file_metadata.file_name = prediction_result.file_name;



DESC TABLE resume_prediction_json_test;




//run the resume presion json file
SELECT top 5* FROM resume_prediction_json_test;

//resume_prediction:"full_name"[0]::variant:"value"::string AS full_name

SELECT DISTINCT
    extraction_timestamp::STRING AS extraction_timestamp,
    RESUME_PREDICTION:"full_name":"value" AS full_name,
    RESUME_PREDICTION:"technical_skills":"value" AS technical_skills,
FROM resume_prediction_json_test;

//using array_slice to take number of specific value index/element 
SELECT DISTINCT
    extraction_timestamp::STRING AS extraction_timestamp,
    ARRAY_SLICE(RESUME_PREDICTION:"technical_skills", 0, 20) AS technical_skills
FROM resume_prediction_json_test;


//semi-structure table with json object 
CREATE OR REPLACE TABLE resume_raw_data AS
SELECT DISTINCT
    extraction_timestamp::STRING AS extraction_timestamp,
    RESUME_PREDICTION:"__documentMetadata" AS document_metadata,
    RESUME_PREDICTION:"full_name" AS full_name,
    RESUME_PREDICTION:"phone_number" AS phone_number,
    RESUME_PREDICTION:"email_address" AS email_address,
    RESUME_PREDICTION:"location" AS location,
    RESUME_PREDICTION:"job_title" AS job_title,
    RESUME_PREDICTION:"work_experience_roles" AS work_experience_roles,
    RESUME_PREDICTION:"work_experience_summary" AS recent_work_experience,
    RESUME_PREDICTION:"technical_skills" AS technical_skills,
    RESUME_PREDICTION:"year_of_experiences" AS year_of_experiences,
    RESUME_PREDICTION:"degree" AS degree,
    RESUME_PREDICTION:"field_of_study" AS field_of_study,
    RESUME_PREDICTION:"university_name" AS university_name,
    RESUME_PREDICTION:"certifications" AS certifications,
    RESUME_PREDICTION:"languages_spoken" AS languages_spoken,
FROM resume_prediction_json_test;

SELECT DISTINCT
    extraction_timestamp::STRING AS extraction_timestamp,
    RESUME_PREDICTION:"full_name" AS full_name,
    ARRAY_AGG(RESUME_PREDICTION:"technical_skills"[0]:"value") AS technical_skills
FROM resume_prediction_json_test
GROUP BY extraction_timestamp, full_name;


SELECT *
FROM resume_prediction;



SELECT RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_SCREENING!PREDICT(
  GET_PRESIGNED_URL(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOCS, RELATIVE_PATH), 1)
FROM DIRECTORY(@RESUME_SCREENING_DB.RESUME_AI_SCHEMA.RESUME_DOC);

